In [89]:
using Plots
using LaTeXStrings
using Printf
using BoundaryValueDiffEq

In [90]:
struct BVPParams
    domain::Tuple{Float64, Float64}
    ϵ::Vector{Float64}  # Can hold multiple values of epsilon
    α::Int
    β::Int
end

In [91]:
# Differential equation definition
function prob2!(dv, v, x, ϵ)
    @views dv[1] = v[2]
    @views dv[2] = (v[2] + 2x) / ϵ  # Solve for the current epsilon
    return dv
end

prob2! (generic function with 1 method)

In [92]:
# Boundary conditions
function boundary_conditions!(residual, u, params::BVPParams, x)
    residual[1] = u[1][1] - params.α
    residual[2] = u[end][1] - params.β
    return residual
end

boundary_conditions! (generic function with 1 method)

In [93]:
# Solve for each epsilon
function solve_bvp(params::BVPParams, idx)
    ϵ = (params.ϵ)[idx]
    # Define the problem and solve for the current epsilon
    initial_guess = [params.α, 0.0]
    bvp = BVProblem(
        (dv, v, p, x) -> prob2!(dv, v, x, ϵ),  # Closure for the current epsilon
        boundary_conditions!,
        initial_guess,
        params.domain,
        params
    )
    dt = 0.001
    sol = solve(bvp, MIRK4(); dt)
    return sol
end

solve_bvp (generic function with 2 methods)

In [94]:
# Plot the solutions
function plot_bvp(sol, ϵ)
    # Initialize an empty plot
    theme(:dao::Symbol)
    plt = plot(
        xlabel = L"$x$",  # Explicitly set the x-axis label
        ylabel = L"$y(x)$",
        title = "Problem 2 Solution "*L"$(\varepsilon=%$ϵ)$",
        legend = :best,
        titlefontsize = 20,
        tickfontsize = 12,
        legendfontsize = 10,
        yguidefontsize = 15,
        xguidefontsize = 15,
        right_margin = 2 * Plots.mm
    )

    plot!(plt,
        sol,
        idxs = [1],
        label = "Exact",
        line = (3, :dash),
        xlabel = L"$x$"  # Ensure the x-axis label is overridden for this plot
    )
    return plt
end

plot_bvp (generic function with 2 methods)

In [ ]:
# Main function
function main()
    domain = (0.0, 1.0)
    α = 1
    β = 2
    ϵ1 = 0.1
    ϵ2 = 0.01

    inner(x, ϵ) = β .+ (β.-α.+1).*(exp.((x .- 1) ./ ϵ) .- 1)
    outer(x, ϵ) = α .- x.^2;
    match(x, ϵ) = 0

    params = BVPParams(domain, [ϵ1, ϵ2], α, β)
    points = collect(LinRange(domain[1], domain[2], 1000))

    exact1 = solve_bvp(params, 1)
    inner1 = inner(points, ϵ1)
    outer1 = outer(points, ϵ1)
    comp1 = inner1 .+ outer1
    comp2 = inner2 .+ outer2 - match

    exact2 = solve_bvp(params, 2)
    inner2 = inner(points, ϵ2)
    outer2 = outer(points, ϵ2)

    fullplot1 = plot_bvp(exact1, ϵ1) 
    plot!(fullplot1, points, [inner1 outer1], 
        line = (3, :solid), 
        labels=["Inner" "Outer"],
        )

    fullplot2 = plot_bvp(exact2, ϵ2) 
    plot!(fullplot2, points, [inner2 outer2], 
        line = (3, :solid), 
        labels=["Inner" "Outer"],
        )

    # Save the combined plot
    savefig(fullplot1, "Aero_HW6_Prob2_eps1.pdf")
    savefig(fullplot2, "Aero_HW6_Prob2_eps2.pdf")
end

main (generic function with 1 method)

In [96]:
main()

"/home/belster12/JuliaFiles/Aero/hw6/Aero_HW6_Prob2_eps2.pdf"